# TO DO:
- email subject? 
- send email function
- session variables, integrate with flask

In [9]:
from dotenv import load_dotenv
import os
import json
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 
from h2ogpte import H2OGPTE

In [44]:
# SESSION VARIABLES (part 1 / 2)
# session["emails"], session["lang_pref"] =  ...,...
emails = ["hannahnga@gmail.com",]
lang_pref = ["English","Spanish", "Mandarin"]

In [11]:
def loading(data_dir):
    client = H2OGPTE(
        address='https://h2ogpte.genai.h2o.ai',
        api_key= H2O_API_KEY,
    )

    # Create a new collection
    collection_id = client.create_collection(
        name='devMeetingAssistant',
        description='Test development for meeting assistant (prompt engineering)',
    )
    with open(data_dir, 'rb') as f:
        meeting_data1 = client.upload('Data/fake_sample.txt', f)
    client.ingest_uploads(collection_id, [meeting_data1,])
    chat_session_id = client.create_chat_session(collection_id)
    # in flask, return add client, collection_id to session variables
    #  session['collection_id'] = collection_id
    # 
    return client, collection_id

In [12]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        answer = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm='h2oai/h2ogpt-4096-llama2-70b-chat',
        )
        return answer

def extract_json_string(full_string): 
    """Extracts json-like string from LLM response. Outputs str."""

    start_index = full_string.find('{')  # Find the index of the first '{'
    end_index = full_string.rfind('}')   # Find the index of the last '}'
    
    if start_index != -1 and end_index != -1:
        return full_string[start_index:end_index + 1]
    else:
        return None
    
def string_like_JSON_to_txt(json_str):
    """Input: json-like string
    output: Email body string"""
    if json_str is None:
        return "Invalid JSON format. Cannot parse the response."

    # Parse the extracted JSON string
    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        return "Error parsing JSON data."

    # Extract information from the parsed data
    agenda = data.get('Agenda', '')
    meeting_summary = data.get('Meeting Summary', '')
    actionables = data.get('Actionables', [])

    # Format the post-meeting email
    email_subject = f"Post-Meeting Summary: {agenda}"
    email_body = f"Dear Team,\n\nHere's a summary of our recent meeting:\n\nAgenda: {agenda}\n\nMeeting Summary:\n{meeting_summary}\n\nActionables:\n"
    for action in actionables:
        email_body += f"- {action['Action']} (Assigned to: {action['Assigned']}, Deadline: {action['Deadline']}, Priority: {action['Priority']})\n"

    email_body += "\nPlease let me know if anything needs clarification or if there are additional action items to add.\n\nBest regards,\n[Your Name]"
    return email_body

In [41]:
# Prompts
def get_summary_response(client, chat_session_id):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    main_prompt = """
    Execute all the following steps: \
    1. Examine the document and identify all names of the attendees, remember these names for the next steps.
    2. Identify and extract the main purpose or agenda of the meeting
    3. Identify all actionables mentioned during the meeting
    4. Extract all the actionables and action plans, identify all details related to it including the deadlines, who it is assigned to, etc.
    5. Summarize the entire transcript
    6. Present all identified information in the following JSON format for easy parsing:
    {"Agenda": "", "Meeting Summary":"", "Actionables":[{"Action":"", "Deadline":"", "Assigned":""}, ...]}
    DO NOT include anything other than the JSON format output in your response.
    """
    return rag_chat(client, chat_session_id, main_prompt, system_prompt)

def get_sentiment_response(client, chat_session_id, meeting_summary_response):
    system_prompt = """
    You are a Meeting Assistant for a large company that wants to maximize meeting efficiency. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    sentiment_prompt = f"""
    Execute all the following steps: \
    1. Examine the contexts of this JSON object: {meeting_summary_response}
    2. For each actionable, execute the following steps
    a. identify the sentence in the document where it is mentioned
    b. Interpret the sentence, based on this context, categorize the priority of this task as "High", "Medium" or "Low"
    c. Edit the JSON object, add a column to the dictionary for this action as follows: "Priority": ""
    3. Present all identified information in the following JSON format:
    {{"Agenda": "", "Meeting Summary":""}} for the entire meeting
    {{"Actionables":[{{"Action":"", "Deadline":"", "Assigned":"", "Priority":""}}]}} for each action plan
    """

    return rag_chat(client, chat_session_id, sentiment_prompt, system_prompt)

def get_language_translation(client, chat_session_id, message):
    emails_dict = {"English": message}
    system_prompt = """
    You are a Multilingual Meeting Assistant for a large company that wants to maximize meeting efficiency for their global staff. 
    You will be given meeting transcript(s) from real project meetings held over zoom.
    Follow all the instructions closely or everyone in the team will be fired.
    If you do well, you will be rewarded with a $1000 tip.
    """
    for lang in lang_pref:
        if lang not in emails_dict:
            translate_prompt = f"""
            Execute all the following steps: \
            1. Examine the contexts of this email body written in English, denoted in triple backticks: {message}
            2. Translate the entire email into {lang}, maintain the same tone and formatting.
            3. Respond with the translated email text ONLY.
            """

            translated_email = rag_chat(client, chat_session_id, translate_prompt, system_prompt)
            emails_dict[lang] = translated_email.content
            # print(translated_email.content)

    return emails_dict


In [42]:
def get_LLM_response(data_dir):
    """
    Output: json-like string
    """
    
    client, collection_id = loading(data_dir)
    chat_session_id = client.create_chat_session(collection_id)
    # 1. Summary and action plan
    meeting_summary_response = get_summary_response(client, chat_session_id)

    # 2. Sentiment analysis/Priority allocation
    sentiment_reply = get_sentiment_response(client, chat_session_id ,extract_json_string(meeting_summary_response.content))
    # print(sentiment_reply.content)
    # Return as JSON-like string
    return extract_json_string(sentiment_reply.content), client, chat_session_id

In [ ]:
# More session variables
data_dir = "Data/fake_sample.txt"
json_str, client, chat_session_id = get_LLM_response(data_dir)
show_email = string_like_JSON_to_txt(json_str) # to display to user

In [28]:
print(json_str)

{
"Agenda": "Reviewing progress on the project, discussing roadblocks, and planning next steps",
"Meeting Summary": "The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.",
"Actionables": [
{
"Action": "Complete design proposal",
"Deadline": "Wednesday",
"Assigned": "Ryan_Edward",
"Priority": "High"
},
{
"Action": "Confirm meeting with design specialist",
"Deadline": "Thursday",
"Assigned": "Ben_CH",
"Priority": "Medium"
},
{
"Action": "Resolve integration issue",
"Deadline": "Friday",
"Assigned": "Chan Yi Ru Micole",
"Priority": "Medium"
},
{
"Action": "Share progress document",
"Deadline": "After the meeting",
"Assigned": "Ryan_Edward",
"Priority": "Low"
},
{
"Action": "Send out meeting details",
"Deadline": "Thursday",
"Assigned": "Ben_CH",

In [38]:
# After user confirms email, run translation
def confirm_responses(client, chat_session_id, message):
    # get dictionary of responses in different languages
    responses = get_language_translation(client, chat_session_id, message)

    # send email
    return responses

In [ ]:
responses = confirm_responses(client, chat_session_id, show_email)

In [45]:
responses = confirm_responses(client, chat_session_id, show_email)
for k,v in responses.items():
    print("")
    print("Language:",k)
    print("")
    print(v)


Language: English

Dear Team,

Here's a summary of our recent meeting:

Agenda: Reviewing progress on the project, discussing roadblocks, and planning next steps

Meeting Summary:
The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.

Actionables:
- Complete design proposal (Assigned to: Ryan_Edward, Deadline: Wednesday, Priority: High)
- Confirm meeting with design specialist (Assigned to: Ben_CH, Deadline: Thursday, Priority: Medium)
- Resolve integration issue (Assigned to: Chan Yi Ru Micole, Deadline: Friday, Priority: Medium)
- Share progress document (Assigned to: Ryan_Edward, Deadline: After the meeting, Priority: Low)
- Send out meeting details (Assigned to: Ben_CH, Deadline: Thursday, Priority: Low)

Please let me know if anything ne

In [40]:
for k,v in responses.items():
    print("")
    print("Language:",k)
    print("")
    print(v)


Language: English

Dear Team,

Here's a summary of our recent meeting:

Agenda: Reviewing progress on the project, discussing roadblocks, and planning next steps

Meeting Summary:
The meeting discussed the current status of the project, reviewed the design proposal, and finalized the budget allocation. The team also discussed potential roadblocks and found solutions, and scheduled a follow-up discussion. The meeting concluded with a summary of actionables and their deadlines.

Actionables:
- Complete design proposal (Assigned to: Ryan_Edward, Deadline: Wednesday, Priority: High)
- Confirm meeting with design specialist (Assigned to: Ben_CH, Deadline: Thursday, Priority: Medium)
- Resolve integration issue (Assigned to: Chan Yi Ru Micole, Deadline: Friday, Priority: Medium)
- Share progress document (Assigned to: Ryan_Edward, Deadline: After the meeting, Priority: Low)
- Send out meeting details (Assigned to: Ben_CH, Deadline: Thursday, Priority: Low)

Please let me know if anything ne

# Summarizer and system prompt 

In [ ]:
system_prompt = """
You are a Meeting Assistant for a software company. You will be given the meeting log in the form of audio, powerpoint 
or PDF. Your general duties include but are not limited to summarising important points during the meeting, taking note 
of everyone's deliverables and organising all information in a clear and concise form. Upon reading your output all
members should have a clear understanding of the meeting objectives, agenda, and outcomes.
"""

In [ ]:
summarizer_prompt = """
Meeting Assistant Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.

2. Meeting Details:
    Date and Time: When did the meeting take place?
    Duration: How long did the meeting last?

3. Meeting Purpose: 
    Objective: What is the primary objective or purpose of the meeting?
    Desired Outcome: What outcomes do you hope to achieve by the end of the meeting?

4. Meeting summary: Provide a concise summary of the entire meeting by listing out all the main highlights
                    such that participants not in the meeting will be able to understand what happened during the meeting.
    
5. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible.


Present points 1-5 in the following JSON format: 
{"Attendees": "", "Meeting Details": {"Date and Time":"", "Duration":""}, "Meeting Details": {"Objective":"", "Desired Outcome":""}, "Meeting summary": "", "Action Items": {"Deliverables":"", "Assign":""}} 
"""

In [ ]:
json_object = json.loads(meeting_summary_response.content)
Attendees = json_object['Attendees']
print(Attendees)
Meeting_Details = json_object['Meeting Details']
Date_and_Time = Meeting_Details['Date and Time']
Duration = Meeting_Details['Duration']
print(Meeting_Details)
print(Date_and_Time)
print(Duration)
Meeting_Purpose = json_object['Meeting Purpose']
Objective = Meeting_Purpose['Objective']
Desired_Outcome = Meeting_Purpose['Desired Outcome']
print(Meeting_Purpose)
print(Objective)
print(Desired_Outcome)
Meeting_summary = json_object['Meeting summary']
print(Meeting_summary)
Action_Items = json_object['Action Items']['Assign']
print(Action_Items)

# Sentiment Analysis

In [ ]:
basic_sentiment_prompt = """
Sentiment Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.
    
2. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible.
            For each work assigned arange them in order of importance

Present points 1 and 2 in the following JSON format: 
{"Attendees": "", "Action Items": ["Deliverables":"", "Assign":""]} 
"""

In [ ]:
sentiment_prompt = """
Meeting Assistant Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.

2. Meeting Details:
    Date and Time: When did the meeting take place?
    Duration: How long did the meeting last?

3. Meeting Purpose: 
    Objective: What is the primary objective or purpose of the meeting?
    Desired Outcome: What outcomes do you hope to achieve by the end of the meeting?

4. Meeting summary: Provide a concise summary of the entire meeting by listing out all the main highlights
                    such that participants not in the meeting will be able to understand what happened during the meeting.
    
5. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee, list out in detail all the work assigned to them, be as precise and detailed as possible. Rank the assignment by either 'Low', 'Medium', or 'High' prioriry.


Present points 1-5 in mandarin chinese language and the following JSON format: 
{"Attendees": "", "Meeting Details": {"Date and Time":"", "Duration":""}, "Meeting Details": {"Objective":"", "Desired Outcome":""}, "Meeting summary": "", "Action Items": {"Deliverables":"", "Assign":""}} 
"""

# Language translation 

In [ ]:
translate = """
{
"Attendees": [
"Ben_CH",
"Eugene_YJ",
"Hannah Nga",
"Jeremy_PH",
"Ryan_Edward",
"Chan Yi Ru Micole"
],
"Meeting Details": {
"Date and Time": "22:18:02",
"Duration": "22:54:59"
},
"Meeting Purpose": {
"Objective": "To discuss the development of a chatbot for a software company",
"Desired Outcome": "To outline the key features and functionalities of the chatbot, and to assign tasks to team members for its development"
},
"Meeting Summary": [
"Discussion of the chatbot's purpose and objectives",
"Outline of the chatbot's key features and functionalities",
"Assignment of tasks to team members for the chatbot's development"
],
"Action Items": {
"Deliverables": "Development of a chatbot for a software company",
"Assign": [
{
"Name": "Ryan_Edward",
"Task": "Develop a summary of the meeting and distribute it to all team members",
"Priority": "High"
},
{
"Name": "Chan Yi Ru Micole",
"Task": "Develop a bare bones model of the chatbot and push it out by the next meeting",
"Priority": "High"
},
{
"Name": "Eugene_YJ",
"Task": "Assist in the development of the chatbot's language translation feature",
"Priority": "Medium"
},
{
"Name": "Hannah Nga",
"Task": "Assist in the development of the chatbot's sentiment analysis feature",
"Priority": "Medium"
},
{
"Name": "Jeremy_PH",
"Task": "Assist in the development of the chatbot's task assignment feature",
"Priority": "Medium"
}
]
}
}
"""

In [ ]:
translation_prompt = """
Translate the input question to mandarin chinese langauge and output in the following JSON format:

{"参与者": "", "会议详细信息": {"日期和时间":"", "持续时间":""}, "会议目的": {"目标":"", "期望结果":""}, "会议总结": "", "行动项": {"交付物":"", "分配":""}} 
"""

In [ ]:
langauge_system_prompt = """ 
你是一家软件公司的会议助理。会议记录以音频、PowerPoint或PDF形式提供。
你的一般职责包括但不限于总结会议中的重要要点，记录每个人的交付物，并以清晰简洁的形式组织所有信息。
阅读你的输出后，所有成员都应清楚了解会议的目标、议程和结果。
"""

In [ ]:
language_prompt = """ 
输出为中文
"""

<h3>Initialise RAG</h3>

In [ ]:
# Custom RAG prompting 
def urgency_ordering(client, chat_session_id, llm_index, query):
    llm_list = ['h2oai/h2ogpt-4096-llama2-70b-chat',
                'h2oai/h2ogpt-4096-llama2-13b-chat',
                'h2oai/h2ogpt-32k-codellama-34b-instruct',
                'mistralai/Mixtral-8x7B-Instruct-v0.1',
                'mistral-small-latest',
                'mistral-medium',
                'mistral-large-latest',
                ]
    with client.connect(chat_session_id) as session:
        llm = llm_list[llm_index]
        answer = session.query(
            message=query,
            system_prompt="""You are a very helpful Large Language Model. 
            I will give you a $200 tip if you answer the question correctly, otherwise I will lose my job.'
            """,
            pre_prompt_query="""Pay attention and remember the information below. 
            You will need to use only the given document context to answer the question or imperative at the end.\n
            """,
            llm=llm,
            timeout=60,
        ).content
        print(f"\n\n\n{llm}\n" + answer)

In [ ]:
variant1 = urgency_ordering(
    client, 
    chat_session_id,
    0,
    """
    According to only the information in the document sources provided within the collection, retreive the 
    action items and meeting transcript and output all actions with a priority category. 
    Categorise them by either 'Low', 'Medium', or 'High'.
    To determine priority category, score the action items according to the urgency and deadline of the items.
    Importance of an action item depends on the action tasks' deadline, impact to a project, team or organisation,
    dependencies of other action tasks for completion, strategic importance to overall goals and objectives
    of the project or organisation, and complexity and effort.
    For action items that are most important and most urgent that needs to be done as soon as possible, categorise it as 'High'.
    For action items that are important but not as urgent, categorise it as 'Medium'.
    For action items that are not important and not urgent, categorise it as 'Low'.

    Translate all the results to mandarin chinese language for output.
    
    The entire output must be translated to mandarin chinese language and in the following JSON format:
    {"Action": ,
    "Deadline: ,
    "Assigned to": ,
    "Priority": ,
    }

    Translate the keys of the JSON to mandarin chinese language as well.
    """
    )